In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('/data/gj/models--THUDM--chatglm-6b/snapshots/f83182484538e663a03d3f73647f10f89878f438', trust_remote_code=True, revision='')

/data/gj/miniconda3/envs/nlpcc/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
a = 'gao jun'
b = 'cool'

a_ids = tokenizer.encode(a, add_special_tokens=False)
b_ids = tokenizer.encode(b, add_special_tokens=False)

In [8]:
a_ids, b_ids

([5, 44106, 5, 13715], [2860])

In [9]:
input_ids = tokenizer.build_inputs_with_special_tokens(a_ids, b_ids)

In [10]:
input_ids

[5, 44106, 5, 13715, 130001, 130004, 2860, 130005]

In [1]:
' '.join(['[MASK]'] * 5)

'[MASK] [MASK] [MASK] [MASK] [MASK]'

In [13]:
tokenizer(a), tokenizer(b)

({'input_ids': [5, 44106, 5, 13715, 130001, 130004]},
 {'input_ids': [2860, 130001, 130004]})

In [ ]:
def preprocess_function_train(examples):
        max_seq_length = data_args.max_source_length + data_args.max_target_length

        model_inputs = {
            "input_ids": [],
            "labels": [],
        }
        for i in range(len(examples[prompt_column])):
            if examples[prompt_column][i] and examples[response_column][i]:
                query, answer = examples[prompt_column][i], examples[response_column][i]

                if history_column is None:
                    prompt = query
                else:
                    prompt = ""
                    history = examples[history_column][i]
                    for turn_idx, (old_query, response) in enumerate(history):
                        prompt += "[Round {}]\n问：{}\n答：{}\n".format(turn_idx, old_query, response)
                    prompt += "[Round {}]\n问：{}\n答：".format(len(history), query)

                prompt = prefix + prompt
                a_ids = tokenizer.encode(text=prompt, add_special_tokens=False)
                b_ids = tokenizer.encode(text=answer, add_special_tokens=False)

                if len(a_ids) > data_args.max_source_length - 1:
                    a_ids = a_ids[: data_args.max_source_length - 1]

                if len(b_ids) > data_args.max_target_length - 2:
                    b_ids = b_ids[: data_args.max_target_length - 2]

                input_ids = tokenizer.build_inputs_with_special_tokens(a_ids, b_ids)

                context_length = input_ids.index(tokenizer.bos_token_id)
                mask_position = context_length - 1
                labels = [-100] * context_length + input_ids[mask_position+1:]
                
                pad_len = max_seq_length - len(input_ids)
                input_ids = input_ids + [tokenizer.pad_token_id] * pad_len
                labels = labels + [tokenizer.pad_token_id] * pad_len
                if data_args.ignore_pad_token_for_loss:
                    labels = [(l if l != tokenizer.pad_token_id else -100) for l in labels]

                model_inputs["input_ids"].append(input_ids)
                model_inputs["labels"].append(labels)

        return model_inputs

In [12]:
import torch

a = torch.randn((2, 3, 4))

In [10]:
b = torch.tensor([[0, 1, 1], [1, 0, 1]])

In [13]:
a

tensor([[[-0.5497, -0.3961,  0.3590,  1.8370],
         [-0.7044, -1.0978,  0.5985,  0.5010],
         [ 1.7006, -1.3426,  1.2077, -0.5090]],

        [[-1.1346,  0.6506,  0.1209,  0.0313],
         [-1.7656, -0.5778,  0.3783,  0.8607],
         [ 1.0308, -0.9678, -1.5656,  1.2382]]])

In [16]:
a[b==1, :].view(a.shape[0], -1, a.shape[-1])

tensor([[[-0.7044, -1.0978,  0.5985,  0.5010],
         [ 1.7006, -1.3426,  1.2077, -0.5090]],

        [[-1.1346,  0.6506,  0.1209,  0.0313],
         [ 1.0308, -0.9678, -1.5656,  1.2382]]])

In [1]:
a = [1,2,3]
a.extend([3,4])

In [2]:
a

[1, 2, 3, 3, 4]

In [6]:
tokenizer.encode('\n答：', add_special_tokens=False)

[5, 4, 67342, 12]

In [11]:
tokenizer.decode(12)

':'

In [27]:
tokenizer.decode(130000, clean_up_tokenization_spaces=False, skip_special_tokens=True)

''

In [38]:
tokenizer.encode('[MASK][', add_special_tokens=False)

[130000, 130000, 5, 9, 9, 10, 13, 9, 10, 13]

In [3]:
from transformers import AutoTokenizer

AttributeError: partially initialized module 'charset_normalizer' has no attribute 'md__mypyc' (most likely due to a circular import)

In [18]:
import torch.nn as nn

a = nn.Linear(4, 5)
b = nn.Linear(5, 4)

In [19]:
import torch
i = torch.randn([2, 4], requires_grad=True)

In [20]:
for p in b.parameters():
    p.requires_grad = False

In [21]:
with torch.no_grad():
    t = a(i)
out = b(t)

In [22]:
out

tensor([[-0.4509,  0.2804, -0.6127, -0.9772],
        [-0.3938,  0.4236, -0.2095, -0.7685]])

In [23]:
out.sum().backward()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn